In [1]:
# import plotly.express as px
# import plotly.io as pio
# import plotly.graph_objects as go
# pio.renderers.default = "notebook_connected" # or use "browser" if you want plots to open with browser
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import einops
from fancy_einsum import einsum
from typing import List, Optional, Callable, Tuple, Union
import functools
from tqdm import tqdm
from IPython.display import display
import random
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
import json


from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

In [2]:
from src.feature_detection.experiment import feature_detection_experiment

In [3]:
gpt2_small = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
# Create the baseline dataset

def read_all_text_files(directory):
    # List to hold the contents of all files
    contents_list = []

    # List all files in directory
    for filename in os.listdir(directory):
        # Check if file is a text file
        if filename.endswith('.txt'):
            # Construct full file path
            filepath = os.path.join(directory, filename)

            # Open the file and read the contents
            with open(filepath, 'r') as f:
                contents = f.read()

            # Add the file contents to the list
            contents_list.append(contents)

    return contents_list

training_subset = read_all_text_files('datasets/urlsf_subset01-1_data') + read_all_text_files('datasets/urlsf_subset01-182_data')



In [5]:
len(training_subset)

599

In [6]:
small_training_subset = training_subset[:100]

In [7]:
stories = {}
# Specify the file path
file_path = 'datasets/fantasy_200.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
  # Load the JSON data from the file
  dataset_fantasy = json.load(file)

  stories["fantasy"] = dataset_fantasy

# Specify the file path
file_path = 'datasets/scifi_200.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
  # Load the JSON data from the file
  dataset_scifi = json.load(file)

  stories["scifi"] = dataset_scifi

# Specify the file path
file_path = 'datasets/sports_200.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
  # Load the JSON data from the file
  dataset_sports = json.load(file)

  stories["sports"] = dataset_sports

In [11]:
dataset_scifi[79]

'In the year 2054, on a remote planet millions of light-years away, a team of astrophysicists stumbled upon an incredible phenomenon: a tear in the fabric of space-time. As they cautiously approached, a blinding light engulfed them, transporting the team to an alternate reality. They soon realized that in this dimension, humans possessed unimaginable powers. Determined to protect their discovery, they formed an intergalactic alliance, safeguarding the portal from those seeking to exploit it. As they continued their exploration, they discovered parallel universes teeming with life, sparking infinite possibilities for the future of humanity.'

In [8]:
feature_dataset = stories["fantasy"][:100] 
baseline_dataset = small_training_subset
evaluation_dataset = {
    "fantasy": stories["fantasy"][100:200],
    "scifi": stories["scifi"][100:200],
    "sports": stories["sports"][100:200]
}

feature_detection_experiment(
    gpt2_small,
    "blocks.7.hook_resid_post",
    feature_dataset,
    baseline_dataset,
    False,
    evaluation_dataset,
    "dot",
    "zero"
)
    

KeyboardInterrupt: 